In [57]:
import os
import csv
import pickle
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from queries import Simbad
from thesis_tools.utilities import calc_r_dist


def get_ang_sep(ra1,dec1,ra2,dec2):
    ra1_rad=ra1*np.pi/180.
    dec1_rad=dec1*np.pi/180.
    ra2_rad=ra2*np.pi/180.
    dec2_rad=dec2*np.pi/180.

    del_ra=ra2_rad-ra1_rad
    del_ras=np.sin(del_ra)
    del_rac=np.cos(del_ra)
    dec1s=np.sin(dec1_rad)
    dec2s=np.sin(dec2_rad)
    dec1c=np.cos(dec1_rad)
    dec2c=np.cos(dec2_rad)
    arctan_arg=np.sqrt((dec2c**2)*(del_ras**2)+(dec1c*dec2s-dec1s*dec2c*del_rac)**2)/(dec1s*dec2s+dec1c*dec2c*del_rac)
    if arctan_arg < 0:
        arctan_arg = -arctan_arg
    return (180./np.pi)*np.arctan(arctan_arg)

x1d_dir='/Users/codydirks/PGCC/x1d_files/'

In [44]:
with open('primary_sightlines.csv','rb') as csvfile:
    inp=csv.reader(csvfile)
    primary_sightlines=inp.next()
    
sightlines=[x for x in os.listdir(x1d_dir) if os.path.isdir(x1d_dir+x) and x.startswith('.')==False]

pgcc_data=fits.open('/Users/codydirks/PGCC/HFI_PCCS_GCC_R2.02.fits')[1].data

In [12]:
pgcc_results=[]
for sightline in sightlines:
    nearest_pgcc=[]
    nearest=999
    ra,dec=Simbad.Position(sightline)
    galcoords=SkyCoord(ra=ra,dec=dec,frame='icrs').galactic
    glon=galcoords.l.value
    glat=galcoords.b.value
    for pgcc in pgcc_data:
        if abs(pgcc['glon']-glon)<1 and abs(pgcc['glat']-glat)<1:
            angsep=get_ang_sep(pgcc['ra'],pgcc['dec'],ra.value,dec.value)
            if angsep<nearest:
                nearest=angsep
                nearest_pgcc=pgcc
    pgcc_results.append([[sightline,ra,dec],nearest_pgcc])

In [17]:
gaia_results=pickle.load(open('gaia_dr2_results.pkl','r'))
gaia_results=[x for x in gaia_results if np.isnan(x[1][0])==False]

84

In [26]:
filt_pgcc_results=[x for x in pgcc_results if x[1]['dist']!=0.0]

61

In [73]:
thres=1.

gaia_sls=[x[0] for x in gaia_results]
for [sl,ra,dec],pgcc in filt_pgcc_results:
    if sl in gaia_sls:
        gaia_data=[x for x in gaia_results if x[0]==sl][0]
        plx,plx_err=gaia_data[1]
        
        if pgcc['dist']<0:
            max_pgcc_dist=round(1000.*abs(pgcc['dist']),2)
            min_pgcc_dist=0.
        else:
            max_pgcc_dist=round(1000*(pgcc['dist']+thres*pgcc['dist_sig']),2)
            min_pgcc_dist=round(1000*(pgcc['dist']-thres*pgcc['dist_sig']),2)
        
        min_star_dist=round(1000./(plx+thres*plx_err),2)
        max_star_dist=round(1000./(plx+thres*plx_err),2)
        if max_star_dist<0:
            max_star_dist=25000.
        if max_pgcc_dist<min_star_dist and sl not in primary_sightlines:
            ra,dec=Simbad.Position(sl)
            print '{:>10}'.format(sl), '{:>5.2f}'.format(round(calc_r_dist(pgcc,ra.value,dec.value),2)),
            print '{:>7.1f}'.format(max_pgcc_dist),'{:>6.1f}'.format(min_star_dist)

  HD108002  8.52   830.0 1811.5
  HD208440  1.23   990.0 1068.7
   HD13268  3.41   800.0 1168.8
